In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import time
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
data = pd.read_csv("Metro_Interstate_Traffic_Volume.csv")
data.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918


In [3]:
#Keeping only traffic_volume data
training_set = data.iloc[:,8:9].values
print(training_set)

[[5545]
 [4516]
 [4767]
 ...
 [2159]
 [1450]
 [ 954]]


In [4]:
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
print(training_set_scaled)

[[0.76167582]
 [0.62032967]
 [0.65480769]
 ...
 [0.29656593]
 [0.19917582]
 [0.13104396]]


In [5]:
timesteps = 60
X_train = []
y_train = []
for i in range(timesteps, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [6]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))

In [7]:
print(X_train.shape)
print(y_train.shape)

(48144, 60, 1)
(48144,)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.05)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(45736, 60, 1) (2408, 60, 1) (45736,) (2408,)


In [9]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=15, batch_size=128)

W1225 16:06:54.723222 20220 deprecation_wrapper.py:119] From c:\users\arc\anaconda3\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/15
45736/45736 [==============================] - 57s 1ms/step - loss: 0.0438
Epoch 2/15
45736/45736 [==============================] - 55s 1ms/step - loss: 0.0142
Epoch 3/15
45736/45736 [==============================] - 55s 1ms/step - loss: 0.0108
Epoch 4/15
45736/45736 [==============================] - 56s 1ms/step - loss: 0.0100
Epoch 5/15
45736/45736 [==============================] - 56s 1ms/step - loss: 0.0091
Epoch 6/15
45736/45736 [==============================] - 55s 1ms/step - loss: 0.0084
Epoch 7/15
45736/45736 [==============================] - 55s 1ms/step - loss: 0.0078
Epoch 8/15
45736/45736 [==============================] - 56s 1ms/step - loss: 0.0074
Epoch 9/15
45736/45736 [==============================] - 55s 1ms/step - loss: 0.0071
Epoch 10/15
45736/45736 [==============================] - 55s 1ms/step - loss: 0.0068
Epoch 11/15
45736/45736 [==============================] - 55s 1ms/step - loss: 0.0066
Epoch 12/15
45736/45736 [===========================

In [10]:
real_values = sc.inverse_transform(y_test.reshape(-1,1))
predicted_values = sc.inverse_transform(model.predict(X_test))

print(model.evaluate(X_test, y_test))

plt.plot(real_values, color='red', label='Real Traffic')
plt.plot(predicted_values, color='blue', label='Predicted Traffic')
plt.title('Traffic Prediction')
plt.legend()
plt.show()                                                      

2408/2408 [==============================] - 5s 2ms/step
0.0050051119812823015
